# U-Net ile Beyin MR Segmentasyonu ve F1-Score Ölçümü

In [ ]:
!pip install nibabel kagglehub tensorflow scikit-learn -q

In [ ]:

import os, numpy as np, nibabel as nib, matplotlib.pyplot as plt, random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:

import kagglehub
path = kagglehub.dataset_download("smitgandhi2005/isles-dataset")
base_path = os.path.join(path, "ISLES-2022", "ISLES-2022", "Training")

def load_case_with_mask(case_id):
    case_path = os.path.join(base_path, case_id)
    adc = nib.load(os.path.join(case_path, f"{case_id}_adc.nii")).get_fdata()
    dwi = nib.load(os.path.join(case_path, f"{case_id}_dwi.nii")).get_fdata()
    flair = nib.load(os.path.join(case_path, f"{case_id}_flair.nii")).get_fdata()
    mask = nib.load(os.path.join(case_path, f"{case_id}_mask.nii")).get_fdata()
    return np.stack([adc, dwi, flair], axis=-1), mask


In [ ]:

def unet_model(input_shape):
    inputs = layers.Input(input_shape)
    def conv_block(x, filters):
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        return x
    def encoder_block(x, filters):
        f = conv_block(x, filters)
        p = layers.MaxPooling2D((2, 2))(f)
        return f, p
    def decoder_block(x, conv_output, filters):
        x = layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
        x = layers.concatenate([x, conv_output])
        x = conv_block(x, filters)
        return x
    c1, p1 = encoder_block(inputs, 32)
    c2, p2 = encoder_block(p1, 64)
    c3, p3 = encoder_block(p2, 128)
    bn = conv_block(p3, 256)
    d1 = decoder_block(bn, c3, 128)
    d2 = decoder_block(d1, c2, 64)
    d3 = decoder_block(d2, c1, 32)
    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(d3)
    return models.Model(inputs, outputs)


In [ ]:

def prepare_training_data(case_ids, max_slices_per_case=5):
    X, Y = [], []
    for case_id in case_ids:
        vol, mask_vol = load_case_with_mask(case_id)
        depth = vol.shape[2]
        slices = np.linspace(0, depth-1, max_slices_per_case, dtype=int)
        for i in slices:
            img = vol[:, :, i, :]
            mask = mask_vol[:, :, i]
            if np.max(mask) < 1e-4:
                continue
            img = img / np.max(img)
            X.append(img)
            Y.append(mask[..., np.newaxis])
    return np.array(X), np.array(Y)


In [ ]:

case_ids = [f"{i:03d}" for i in range(1, 21)]
X, Y = prepare_training_data(case_ids)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

model = unet_model(input_shape=(x_train.shape[1], x_train.shape[2], 3))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=4)


In [ ]:

def plot_unet_predictions(model, image_batch, true_masks=None, num_samples=5):
    preds = model.predict(image_batch[:num_samples])
    for i in range(num_samples):
        fig, ax = plt.subplots(1, 3 if true_masks is not None else 2, figsize=(12, 4))
        ax[0].imshow(image_batch[i, :, :, 1], cmap='gray')
        ax[0].set_title("Girdi (DWI)")
        ax[0].axis('off')
        if true_masks is not None:
            ax[1].imshow(true_masks[i, :, :, 0], cmap='gray')
            ax[1].set_title("Gerçek Maske")
            ax[1].axis('off')
        ax_idx = 2 if true_masks is not None else 1
        ax[ax_idx].imshow(preds[i, :, :, 0], cmap='hot')
        ax[ax_idx].set_title("Tahmini Maske")
        ax[ax_idx].axis('off')
        plt.tight_layout()
        plt.show()

y_pred = model.predict(x_val)
y_pred_bin = (y_pred > 0.5).astype(np.uint8)
y_val_bin = (y_val > 0.5).astype(np.uint8)
f1 = f1_score(y_val_bin.flatten(), y_pred_bin.flatten())
print("Validation F1-Score:", f1)

plot_unet_predictions(model, x_val, true_masks=y_val, num_samples=5)
